In [29]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
from transformers import AutoModelForCausalLM
import torch
qwen_model_name = "Bossologist/Qwen3-4B-Instruct-2507_general_ft_merged"

qwen_model = AutoModelForCausalLM.from_pretrained(
    qwen_model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
)


c:\Users\Charlie\anaconda3\envs\ai_assistant\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1225 08:22:58.955000 38644 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [ ]:
from perceiver_module import PerceiverIOModule, CrossAttentionCompressor

latent_dim = 512

perceiver_module = PerceiverIOModule(
    input_dim=qwen_model.config.hidden_size,
).to(device)

compressor = CrossAttentionCompressor(
    latent_dim=latent_dim,
    num_heads=8,
).to(device)

Created input projection: 2560 -> 704


In [31]:
from transformers import AutoTokenizer

tokenizer_path = qwen_model_name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)

In [32]:
input = "Hello there my friend, how are you?"
encoded = tokenizer(
    input,
    max_length=2048,
    truncation=True,
    padding=True,
    return_tensors="pt",
)
input_ids = encoded["input_ids"].to(device)
attention_mask = encoded["attention_mask"].to(device)
input_ids.shape

torch.Size([1, 9])

In [33]:
embed_layer = qwen_model.get_input_embeddings()
turn_embeddings = embed_layer(input_ids).to(device)
turn_embeddings

tensor([[[-0.0151, -0.0030, -0.0091,  ..., -0.0067,  0.0154,  0.0013],
         [-0.0060,  0.0098,  0.0160,  ...,  0.0072,  0.0408,  0.0229],
         [-0.0277,  0.0269,  0.0002,  ...,  0.0012,  0.0155, -0.0036],
         ...,
         [-0.0181,  0.0013, -0.0035,  ..., -0.0305,  0.0153,  0.0032],
         [-0.0208,  0.0033,  0.0082,  ...,  0.0126,  0.0127,  0.0305],
         [-0.0168, -0.0410,  0.0051,  ..., -0.0031, -0.0148,  0.0481]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<EmbeddingBackward0>)

In [38]:
outputs = perceiver_module(turn_embeddings)
outputs[0][0].shape

torch.Size([512])